# Convert EDF to expipe files
The visual EEG inspection table 'eeg-sichtung.xlsx' has been manually translated into a form which is more easy to read automatically.
The table is located in the new file 'data_annotation.ods'.
It contains one row per day an animal is recorded.
Relevant metadata such as animal id, recording channel, artifacts or missing data are stored in separate columns.

This notebooks loads the original edf files sequentially and converts the recordings to the expipe format [1].
Here, each day an animal is recorded is its own action.

[1] https://github.com/CINPLA/expipe

In [1]:
import numpy as np
import pandas as pd
from tempfile import mkdtemp
import zipfile
import shutil
from tqdm import tqdm
import expipe
import pyedflib
import datetime
import os
import tools
from format_data_annotation import dict_format
import pdb

In [2]:
#path = ''
path = '/home/stoeber/EpimiRNA_ExpipeConversion/'
fname = 'data_annotation.ods'

In [3]:
# Info about the EEG files
#path_edf = "/home/jovyan/work/raw_files/"
path_edf = "/home/epilepsy-data/data/PPS_data_raw_transfer/"

#path_expipe = "/home/jovyan/work/data_expipe/epimirna"
path_expipe = "/home/epilepsy-data/data/epimirna/"

# decide whether existing data should be rebuild or not
bool_keepexisting = True

### Read data annotation

In [4]:
t = pd.read_excel(path+fname, engine="odf", dtype=dict_format)
#t = pd.read_excel(path+fname, engine="odf")

In [5]:
t['ID'].unique()

array([ 1218,  1227,  1237,  1243,  1251,  1254,  1257,  1260,  1261,
        1262,  1263,  1264,  1265,  1269,  1270,  1271,  1274,  1275,
        1276,   322,   323,   324,   325,   326,   328,   329,  3210,
        3211,  3212,  3215,  3216,  3218, 32220,  3222, 32223,  3224,
        3226,  3227,  3228,  3231,  3232,  3233,  3235,  3237,  3238,
        3239,  3240,  3241,  3242,  3243,  3244,  3245,  3246,  3247,
        3248,  3251,  3253,  3254,  3256,  3258,  3259,  3260,  3261,
        3262,  3263,  3266,  3267,  3268,  3269,  3270,  3271,  3272,
        3273,  3274,  3275,  3276,  3277,  3278,  3279,  3280,  3281,
        3282,  3283,  3284,  3285])

### Create Expipe project and store metadata for each animal

In [6]:
animals = t['ID'].unique()
print(animals)

[ 1218  1227  1237  1243  1251  1254  1257  1260  1261  1262  1263  1264
  1265  1269  1270  1271  1274  1275  1276   322   323   324   325   326
   328   329  3210  3211  3212  3215  3216  3218 32220  3222 32223  3224
  3226  3227  3228  3231  3232  3233  3235  3237  3238  3239  3240  3241
  3242  3243  3244  3245  3246  3247  3248  3251  3253  3254  3256  3258
  3259  3260  3261  3262  3263  3266  3267  3268  3269  3270  3271  3272
  3273  3274  3275  3276  3277  3278  3279  3280  3281  3282  3283  3284
  3285]


In [7]:
# Open project
project = expipe.require_project(path_expipe)

for animal_i in animals:
    
    # Register each animal
    entity_i = project.require_entity(str(animal_i))
    t_i = t[t['ID']==animal_i]
    entity_i.modules['info'] = {
        'channel': t_i['Channel'].unique()[0],
        'machine': t_i['Machine'].unique()[0]
    }
    # Create action for each animal
    name_action_i = 'record_'+str(animal_i)
    action_i = project.require_action(name_action_i)
    
    action_i.type = "eeg recording"
    action_i.entities = [str(animal_i)]
    
    # store sub-table for each animal
    fname_annot = 'df_data_annotation.pckl'
    t_i.to_pickle(action_i.data_path().joinpath(fname_annot))
    action_i.data['df_data_annotation'] = fname_annot

### Add concise group names for the whole epimirna dataset

In [79]:
from annotation_short import annotation as group_names_epimirna
print(group_names)

{'1': 'FF_EPG_5Hz', '1_noPPS': 'FF_EPG_5Hz_noPPS', '2': 'FF_EPG_130Hz', '3': 'PP_EPG_5Hz', '4': 'PP_EPG_130Hz', '5': 'DG_EPG_5Hz', '6': 'DG_EPG_130Hz', '8': 'FF_ME_5Hz', '9': 'FF_ME_130Hz', '10': 'PP_EPG_5Hz', '11': 'PP_EPG_130Hz', '12': 'DG_EPG_5Hz', '13': 'DG_EPG_130Hz', '14': 'FF_noDBS', '14_noPPS': 'FF_noDBS_noPPS', '15': 'VHC_noDBS', '16': 'noDBS', 'pps_only': 'noDBS', 'vhc_epilep_1 Hz': 'VHC_EPG_1Hz', 'vhc_epilep_HF': 'VHC_EPG_130Hz', 'vhc_epilep_LF (5 Hz)': 'VHC_EPG_5Hz', 'vhc_epileptog_LF': 'VHC_EPG_?Hz', 'vhc_mE_Kontrolle': 'VHC_noDBS', 'vhc_mE_LF (1 Hz)': 'VHC_ME_1Hz', 'vhc_mE_LF (1Hz)': 'VHC_ME_1Hz', 'vhc_mE_LF (5 Hz)': 'VHC_ME_5Hz', 'vhc_mE_LF (5Hz)': 'VHC_ME_5Hz'}


In [81]:
for animal_i in animals:
    # load
    action_i = project.require_action('record_'+str(animal_id))
    t_i = pd.read_pickle(action_i.data_path().joinpath(fname_annot))
    
    # get epimirna group name, rename it
    group_i = t_i['group'].unique()
    assert len(group_i) == 1
    group_i = group_i[0]

    # define consisten group names across the whole epimirna data set
    t_i['group_epimirna'] = group_names_epimirna[group_i]
    
    # store
    t_i.to_pickle(action_i.data_path().joinpath(fname_annot))
    
    

In [82]:
t_i

,Date,ID,group,Machine,Channel,Notes,EEG manually inspected,PPS Stimulation,Artifacts reported,Surgery,...,Missing data,Conflict overlap recording,Device on,Device off,Spikes,Spindles,DBS location,DBS frequency,DBS stimulation,group_epimirna
7375,2016-06-27,3285,vhc_epilep_1 Hz,yolo,9,"LC (Edf) Anfall 16:22, 17:14, 17:26, Op",x,NaN,NaN,x,...,x,NaN,x,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz
7376,2016-06-28,3285,vhc_epilep_1 Hz,yolo,9,LC (Edf),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz
7377,2016-06-29,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz
7378,2016-06-30,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF) Anfall? 12:21:13 (Cluster) // kein An...,x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz
7379,2016-07-01,3285,vhc_epilep_1 Hz,yolo,9,VN (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7470,2016-09-30,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz
7471,2016-10-01,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz
7472,2016-10-02,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz
7473,2016-10-03,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF) Anfall 14:35:53,x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN,VHC_EPG_1Hz


,Date,ID,group,Machine,Channel,Notes,EEG manually inspected,PPS Stimulation,Artifacts reported,Surgery,...,Seizure,Missing data,Conflict overlap recording,Device on,Device off,Spikes,Spindles,DBS location,DBS frequency,DBS stimulation
7375,2016-06-27,3285,vhc_epilep_1 Hz,yolo,9,"LC (Edf) Anfall 16:22, 17:14, 17:26, Op",x,NaN,NaN,x,...,"16:22, 17:14, 17:26",x,NaN,x,NaN,NaN,NaN,VHC,1,NaN
7376,2016-06-28,3285,vhc_epilep_1 Hz,yolo,9,LC (Edf),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN
7377,2016-06-29,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN
7378,2016-06-30,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF) Anfall? 12:21:13 (Cluster) // kein An...,x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN
7379,2016-07-01,3285,vhc_epilep_1 Hz,yolo,9,VN (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7470,2016-09-30,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN
7471,2016-10-01,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN
7472,2016-10-02,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF),x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN
7473,2016-10-03,3285,vhc_epilep_1 Hz,yolo,9,LC (EDF) Anfall 14:35:53,x,NaN,NaN,NaN,...,14:35:53,NaN,NaN,NaN,NaN,NaN,NaN,VHC,1,NaN


In [67]:
action_i.data_path()

PosixPath('/home/epilepsy-data/data/epimirna/actions/record_3285/data')

In [61]:
project.actions

In [58]:
[action_i for action_i in project.actions]

['record_3215',
 'record_1218',
 'record_3285',
 'record_1260',
 'record_3260',
 'record_3222',
 'record_3266',
 'record_323',
 'record_3232',
 'record_3256',
 'record_3263',
 'record_1263',
 'record_32220',
 'record_3278',
 'record_1274',
 'record_3279',
 'record_3254',
 'record_3241',
 'record_3259',
 'record_3239',
 'record_3274',
 'record_3238',
 'record_1251',
 'record_1262',
 'record_3247',
 'record_3262',
 'record_3218',
 'record_3233',
 'record_3268',
 'record_3224',
 'record_3271',
 'record_3240',
 'record_3251',
 'record_3226',
 'define_epg_phases',
 'record_1270',
 'record_3237',
 'record_3231',
 'record_329',
 'record_1271',
 'record_3243',
 'record_1254',
 'record_325',
 'record_326',
 'record_1264',
 'record_322',
 'record_3272',
 'record_3270',
 'record_1237',
 'record_3253',
 'record_3258',
 'record_3227',
 'record_3242',
 'record_3283',
 'record_3282',
 'record_3248',
 'record_3280',
 'record_3269',
 'record_3246',
 'record_3244',
 'record_1227',
 'record_3267',
 'reco

'record_3285'

In [33]:
action_i.name

AttributeError: 'Action' object has no attribute 'name'

In [48]:
action_i = [action_i for action_i in project.actions if action_i.id == 'record_'+str(animal_i)]

AttributeError: 'str' object has no attribute 'id'

{'1': 'FF_EPG_5Hz',
 '1_noPPS': 'FF_EPG_5Hz_noPPS',
 '2': 'FF_EPG_130Hz',
 '3': 'PP_EPG_5Hz',
 '4': 'PP_EPG_130Hz',
 '5': 'DG_EPG_5Hz',
 '6': 'DG_EPG_130Hz',
 '8': 'FF_ME_5Hz',
 '9': 'FF_ME_130Hz',
 '10': 'PP_EPG_5Hz',
 '11': 'PP_EPG_130Hz',
 '12': 'DG_EPG_5Hz',
 '13': 'DG_EPG_130Hz',
 '14': 'FF_noDBS',
 '14_noPPS': 'FF_noDBS_noPPS',
 '15': 'VHC_noDBS',
 '16': 'noDBS',
 'pps_only': 'noDBS',
 'vhc_epilep_1 Hz': 'VHC_EPG_1Hz',
 'vhc_epilep_HF': 'VHC_EPG_130Hz',
 'vhc_epilep_LF (5 Hz)': 'VHC_EPG_5Hz',
 'vhc_epileptog_LF': 'VHC_EPG_?Hz',
 'vhc_mE_Kontrolle': 'VHC_noDBS',
 'vhc_mE_LF (1 Hz)': 'VHC_ME_1Hz',
 'vhc_mE_LF (1Hz)': 'VHC_ME_1Hz',
 'vhc_mE_LF (5 Hz)': 'VHC_ME_5Hz',
 'vhc_mE_LF (5Hz)': 'VHC_ME_5Hz'}

### Load EEG files and store them as actions

#### Exclude entries with overlapping recordings


In [8]:
t = t[t['Conflict overlap recording']!='x']

#### Test consistency of leftover table

In [9]:
t = t[t['Device off'].isnull() & t['Missing data'].isnull()]
# Test consistency of the table for data extraction
tools.test_consistency_of_annotation_table(t)

In [10]:
t = t[t['Missing data']!='x']

In [ ]:
# Open EDF files by date and machine
grp = t.groupby(['Date', 'Machine'])
for (date_i, machine_i), grp_i in tqdm(grp):
    
    # unzip file to temporary folder
    date_str = date_i.date().strftime('%Y-%m-%d')
    fname = "EDF_" + machine_i + '_' + date_str
    fname_zip = fname + '.zip'
    tmpdir = mkdtemp()
    try:
        with zipfile.ZipFile(path_edf + fname_zip, 'r') as zip_ref:
            zip_ref.extractall(tmpdir)
    except FileNotFoundError as err:
        print(err)
        shutil.rmtree(tmpdir)
        continue

    # load recordings
    fname_edf = fname + '.edf'
    try:
        f = pyedflib.EdfReader(tmpdir + '/' + fname_edf)
    except OSError as err:
        print("OS error: {0}".format(err))
        print(fname_edf + ' not found')
        f.close()
        shutil.rmtree(tmpdir)
        continue
    
    # extract information from recording
    n = f.signals_in_file
    start_time = f.getStartdatetime()
    signal_labels = f.getSignalLabels()
    nsamples = f.getNSamples()
    assert len(np.unique(nsamples)) == 1
    nsamples = np.unique(nsamples)[0]
    
    # associate recording to animal entity
    for label_i in signal_labels:
        pos_i = signal_labels.index(label_i)
        t_ii = grp_i[grp_i['Channel'] == int(label_i)]
        if np.any(t_ii):
            animal_id = t_ii['ID'].unique()
            assert(len(animal_id) == 1)
            animal_id = str(animal_id[0])
            
            # get corresponding action
            action_i = project.require_action('record_'+str(animal_id))
            
            name_rec = 'eeg_recording_' + date_str
            # check if recording already exists
            if (name_rec in list(action_i.data) and
                bool_keepexisting):
                print('Skipped file. Already present: ' + name_rec)
                continue
            else:
                # Get all information from table except date
                info = t_ii.to_dict('records')[0]
                
                # date is removed here, it is stored in a different place
                del info['Date']
                
                # if any entry in info is datetime.time, convert to hours, minutes, seconds
                for k, v in info.items():
                    if isinstance(v, datetime.time):
                        info[k] = v.strftime("%d/%m/%Y, %H:%M:%S")
                
                sampling_freq = f.getSampleFrequency(pos_i)
                info['sampling_freq'] = sampling_freq
                info['nsamples'] = nsamples
                info['duration'] = nsamples*1./sampling_freq
                info['dimension'] = f.getPhysicalDimension(pos_i)
                #info['starttime'] = start_time.strftime("%d/%m/%Y, %H:%M:%S")
                #info['annotations'] = list(t_ii.to_dict('index').values())

                mod_info = action_i.require_module(name=name_rec+'_info', contents=info)

                 # link file in expipe
                action_i.data[name_rec] = name_rec + '.npz'

                # store recordings in separate file
                sig_ii = f.readSignal(pos_i)
                action_data_path = str(action_i.data_path(name_rec))
                np.savez_compressed(action_data_path, eeg=sig_ii)
    # close EDF file
    f.close()
    # delete temporary folder
    shutil.rmtree(tmpdir)


  0%|▏                                                                                                                                                                        | 1/1023 [05:51<99:52:09, 351.79s/it]

Skipped file. Already present: eeg_recording_2014-07-21


  0%|▎                                                                                                                                                                       | 2/1023 [11:50<100:52:14, 355.67s/it]

Skipped file. Already present: eeg_recording_2014-07-22


  0%|▍                                                                                                                                                                       | 3/1023 [18:31<106:38:29, 376.38s/it]

...

 17%|███████████████████████████▉                                                                                                                                         | 173/1023 [6:01:04<39:50:25, 168.74s/it]

Skipped file. Already present: eeg_recording_2015-01-24
Skipped file. Already present: eeg_recording_2015-01-24
Skipped file. Already present: eeg_recording_2015-01-25
Skipped file. Already present: eeg_recording_2015-01-25
Skipped file. Already present: eeg_recording_2015-01-25
Skipped file. Already present: eeg_recording_2015-01-25


 17%|████████████████████████████                                                                                                                                         | 174/1023 [6:03:54<39:51:21, 169.00s/it]

Skipped file. Already present: eeg_recording_2015-01-25
Skipped file. Already present: eeg_recording_2015-01-26
Skipped file. Already present: eeg_recording_2015-01-26


 17%|████████████████████████████▏                                                                                                                                        | 175/1023 [6:06:39<39:35:38, 168.09s/it]